### Installation
Install the packages required for executing this notebook.

In [2]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check the versions of the packages you installed. The KFP SDK version should be >2.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.7.0
google-cloud-aiplatform==1.69.0
google_cloud_pipeline_components version: 2.17.0


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [24]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "your poject id"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://your temp bucket"

#### Pipeline Component : Data Ingestion

In [5]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

#### Pipeline Component : Training-MLP 

In [6]:
@dsl.component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py'],
    base_image="python:3.10.7-slim"
)
def train_mlp (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a MLP with default parameters'''
    import pandas as pd
    from keras.layers import Dense
    from keras.models import Sequential
    import json
    import logging 
    import sys
    import os
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)
        
    # split into input (X) and output (Y) variables
    X = df.loc[:, ['ntp', 'pgc', 'dbp', 'tsft', 'si', 'bmi', 'dpf', 'age']].values
    Y = df.loc[:, ['class']].values
    # define model
    model_mlp = Sequential()
    model_mlp.add(Dense(12, input_dim=8, activation='relu'))
    model_mlp.add(Dense(8, activation='relu'))
    model_mlp.add(Dense(1, activation='sigmoid'))
    # compile model
    model_mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model_mlp.fit(X, Y, epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model_mlp.evaluate(X, Y, verbose=0)
    logging.info(model_mlp.metrics_names)
    metrics_dict = {
        "accuracy": scores[1],
        "loss": scores[0],
    }   
    logging.info(metrics_dict)   
    
    out_model.metadata["file_type"] = ".h5"
    out_model.metadata["algo"] = "mlp"
    # Save the model
    m_file = out_model.path + ".h5"
    model_mlp.save(m_file)
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Training LogisticRegression

In [7]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_lr (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('class',axis=1), 
                                                    df['class'], test_size=0.30, 
                                                    random_state=101)
    model_lr = LogisticRegression()
    model_lr.fit(x_train,y_train)

    metrics_dict = {
        "accuracy": model_lr.score(x_test, y_test)
    }
    logging.info(metrics_dict)  
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "lr"
   # Save the model
    m_file = out_model.path + ".pkl"
    with open(m_file, 'wb') as f:  
        pickle.dump(model_lr, f)   
    
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Prediction-MLP

In [8]:
@dsl.component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py'],
    base_image="python:3.10.7-slim"
)
def predict_mlp(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from keras.models import load_model
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    xNew = df.loc[:, ['ntp', 'pgc', 'dbp', 'tsft', 'si', 'bmi', 'dpf', 'age']].values
    #load the model
    model_mlp = load_model(model.path + '.h5')
    dfcp = df.copy()
    result = model_mlp.predict(xNew)   
    y_classes = result.argmax(axis=-1)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Prediction-LR

In [9]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def predict_lr(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle  
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
       
    filename = model.path + ".pkl"
        
    #Loading the saved model
    model_lr = pickle.load(open(filename, 'rb'))

    xNew = df[['ntp', 'pgc', 'dbp', 'tsft', 'si', 'bmi', 'dpf', 'age']]

    dfcp = df.copy()   
    y_classes = model_lr.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Algorithm Selection 

In [19]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(mlp_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(mlp_metrics)
    logging.info(lr_metrics)
    if mlp_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "MLP"
    else :
        return "LR"

### Upload Model and Metrics to Google Bucket 

In [20]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"])) 
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [21]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="diabetes-prdictor-training-pipeline-v2")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_mlp_job_run_op = train_mlp(
        features=di_op.outputs["dataset"]
    )
    
     
    training_lr_job_run_op = train_lr(
        features=di_op.outputs["dataset"]
    )
    
    pre_di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_mlp_job_run_op, training_lr_job_run_op)
        
        
    comp_model__op = compare_model(mlp_metrics=training_mlp_job_run_op.outputs["metrics"],
                                       lr_metrics=training_lr_job_run_op.outputs["metrics"]).after(training_mlp_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.If(comp_model__op.output=="MLP"):
        predict_mlp_job_run_op = predict_mlp(
            model=training_mlp_job_run_op.outputs["out_model"],      
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_mlp_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_mlp_job_run_op.outputs['out_model']
        ).after(predict_mlp_job_run_op)
        
    with dsl.If(comp_model__op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
            model=training_lr_job_run_op.outputs["out_model"],     
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_lr_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['out_model']
        ).after(predict_lr_job_run_op)  

#### Compile the pipeline into a JSON file

In [22]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='diabetes_predictor_training_pipeline_ex1.yaml')

#### Submit the pipeline run

In [23]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="diabetes-predictor-ex1",
    enable_caching=False,
    template_path="diabetes_predictor_training_pipeline_ex1.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2024',  # makesure to use your data bucket name 
        'trainset_filename': 'training_set.csv',     # makesure to upload these to your data bucket from DE2024/lab4/data
        'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE2024/lab4/data
        'model_repo':'models_de2024' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/diabetes-prdictor-training-pipeline-v2-20241007202913?project=966204339179
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-